In [2]:
%pip install --upgrade pip

%pip install numpy
%pip install pandas
%pip install matplotlib

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [21]:
df_train.describe()


,id,name,intake_time,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,outcome_time,date_of_birth,outcome_type
count,111157,79774,111157,111157,111157,111157,111157,111155,111156,111157,111157,111157,111157,111157
unique,111157,24185,78769,48762,6,19,2,5,48,2440,569,95134,8181,5
top,A706918,Luna,09/23/2016 12:00:00 PM,Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Domestic Shorthair Mix,Black/White,04/18/2016 12:00:00 AM,05/05/2014,Adoption
freq,1,481,49,19071,82163,95010,61378,41060,16192,25361,11620,28,85,55044


In [ ]:
## Data Preprocessing ##
'''
print('records with missing names:', df_train['name'].isna().sum())

print('records that are blank or has numbers:', non_alpha_values.sum()) # does not wrok but you get the idea. fix pls
'''

In [ ]:
# Rebecca's slay data exploration block /ᐠ - ˕ -マ 
# intake time, color, outcome time, date of birth

In [23]:
# Nneoma's data visualation

# I am going to look at the Intake Type
df_train['intake_type'].head(50)


0               Stray
1               Stray
2       Public Assist
3     Owner Surrender
4       Public Assist
5               Stray
6       Public Assist
7               Stray
8               Stray
9     Owner Surrender
10              Stray
11              Stray
12      Public Assist
13              Stray
14    Owner Surrender
15              Stray
16              Stray
17              Stray
18              Stray
19              Stray
20              Stray
21              Stray
22      Public Assist
23    Owner Surrender
24          Abandoned
25              Stray
26              Stray
27              Stray
28              Stray
29    Owner Surrender
30              Stray
31              Stray
32              Stray
33    Owner Surrender
34              Stray
35              Stray
36              Stray
37              Stray
38              Stray
39    Owner Surrender
40              Stray
41              Stray
42    Owner Surrender
43              Stray
44              Stray
45        